$ \mathrm{\overline{C}_{D}}_{t_0}^{t_N}(\mathbf{x}_0) $ is given by:

\begin{equation}
\mathrm{\overline{C}_{D}}_{t_0}^{t_N}(\mathbf{x}_0) = \dfrac{1}{t_N-t_0} \int_{t_0}^{t_N}\mathrm{det}[\mathbf{D}(\mathbf{F}_{t_0}^t(\mathbf{x}_0), t)] [\mathbf{T}_{t_0}^t(\mathbf{x}_0)]^{-1}dt 
\end{equation}

and denotes the time-averaged, diffusive structure-weighted version of the classic right Cauchy–Green strain tensor. 

The transport tensor $ \mathrm{T_{t_0}^{t}}(\mathbf{x}_0) $ is given by:

\begin{equation}
\mathrm{T_{t_0}^{t}}(\mathbf{x}_0) = [\nabla \mathbf{F}_{t_0}^t]^{-1}(\mathbf{x}_0) \mathbf{D}(\mathbf{F}_{t_0}^t(\mathbf{x}_0), t)) [\nabla \mathbf{F}_{t_0}^t]^{-T}(\mathbf{x}_0).
\end{equation}


It thus holds:

\begin{equation}
\mathrm{\overline{C}_{D}}_{t_0}^{t_N}(\mathbf{x}_0) = \dfrac{1}{t_N-t_0} \int_{t_0}^{t_N}\mathrm{det}[\mathbf{D}(\mathbf{F}_{t_0}^t(\mathbf{x}_0), t)] [\nabla \mathbf{F}_{t_0}^t]^{T}(\mathbf{x}_0)\mathbf{D}(\mathbf{F}_{t_0}^t(\mathbf{x}_0), t)^{-1}[\nabla \mathbf{F}_{t_0}^t(\mathbf{x}_0)] dt
\end{equation}

$ \mathbf{D}(\mathbf{x}(t), t) = \mathbf{D}(\mathbf{x}(t), t)^T = \begin{pmatrix} D_{11} && D_{12} \\ D_{12} && D_{22} \end{pmatrix} $ denotes a continuously differentiable, positive definite diffusion-structure tensor, with possible anisotropy and temporal variation. $ \mathbf{D}(\mathbf{x}(t), t) $ must also be an objective tensor. In the following notebooks we will omit spatial and temporal variations of the $ \mathbf{D}(\mathbf{x}(t), t) $ and assume it to be constant. For isotropic diffusion ($ \mathbf{D}(\mathbf{x}(t), t) = \mathbf{I} $) it holds:

\begin{equation}
\mathrm{T_{t_0}^{t}}(\mathbf{x}_0) = [\mathbf{C}_{t_0}^t(\mathbf{x}_0)]^{-1}.
\end{equation}

| Name | Type (Shape) | Description |
| --- | --- | --- |
| Interpolant_D11 | Interpolant object | Interpolant for $ D_{11} $|
| Interpolant_D22 | Interpolant object | Interpolant for $ D_{22} $|
| Interpolant_D12 | Interpolant object | Interpolant for $ D_{12} $|
| time | array (Nt, ) | time |
| Fmap | array (Nt, 2) | flow map |
| gradFmap | array (Nt, 2, 2) | gradient of flow-map |
| lenT | float | $ t_N-t_0 $ |
| n_averaging | float | Number of data points used for averaging |
| CD_avg | array (2, 2) | $ \mathrm{\overline{C}_{D}}_{t_0}^{t_N}(\mathbf{x}_0) $ |

In [2]:
# Import numpy
import numpy as np

In [3]:
def DiffusionCauchyGreen(Interpolant_D11, Interpolant_D12, Interpolant_D22, time, Fmap, gradFmap, lenT, n_averaging = 10):
    '''
    Computed the  time-averaged, diffusive structure-weighted version of the classic right Cauchy–Green strain tensor. 
    
    Parameters:
        Interpolant_D11: Interpolant object, Interpolant for  D_{11}
        Interpolant_D22: Interpolant object, Interpolant for  D_{22}
        Interpolant_D12: Interpolant object, Interpolant for  D_{12}
        time: array (Nt, )  time 
        Fmap: array (Nt, 2)  flow map 
        gradFmap: array (Nt, 2, 2), gradient of flow-map 
        lenT: float, length of time interval
        n_averaging: float, Number of data points used for averaging 
        
    Returns:
        CD_avg: array (2, 2)  \overline{C}_{D}_{t_0}^{t_N}(x_0)
    '''
    # Diffusive Cauchy Green
    Cd = gradFmap.copy()*np.nan # array (2,2)
    
    # averaged Diffusive Cauchy Green 
    Cd_avg = np.zeros((2,2)) # array (2,2)
    
    counter = 0
    
    pts = np.array([Fmap[:, 1].ravel(), Fmap[:, 0].ravel(), time.ravel()]).transpose() # array (Nt, 3)
    
    # evaluate D11 at x(t)
    D11 = Interpolant_D11(pts) # float
    
    # evaluate D12 at x(t)
    D12 = Interpolant_D12(pts) # float
        
    # evaluate D22 at x(t)
    D22 = Interpolant_D22(pts) # float
                                      
    for i in range(0, gradFmap.shape[0], n_averaging):

        # determinant of D
        detD = D11[i]*D22[i]-D12[i]**2 # float
        
        D = np.array([[D11[i], D12[i]], [D12[i], D22[i]]]) # array (2,2)

        # diffusion averaged CG tensor
        Cd[i,:,:] = detD*(gradFmap[i,:,:].transpose()@(D@gradFmap[i,:,:])) # array (2,2)
        
        Cd_avg += Cd[i,:,:] # array (2,2)
        
        counter += 1
    
    Cd_avg = Cd_avg/counter
    
    return Cd_avg